In [59]:
# Input: number of beads; number of probes; box size

# Output:
# 1. Configuration: positions of obstacles (r=2) and probes (r=0)
# 2. Obstacle: id, image id, tetrahedron id
# 3. Tetrahedron: id, image id, if percolation, obstacles id

In [60]:
import math
import numpy as np
import pandas as pd
from scipy.spatial import Voronoi
from scipy.spatial import Delaunay
import networkx as nx
import random
import copy
from collections import defaultdict, Counter
import itertools
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import imageio.v2 as imageio
import os
import warnings
warnings.simplefilter("error")
from types import MappingProxyType

In [61]:
# input parameters
seedNumber = 0
numBeads = 1000
numProbes = 1000
box_size = 100.0

filepath =  str(numBeads) + '_' + str(seedNumber)

np.random.seed(seedNumber)

In [62]:
def replicate_points(points, box_size):
    """Replicate points across periodic boundaries."""
    replicas = []
    for dx in [-box_size, 0, box_size]:
        for dy in [-box_size, 0, box_size]:
            for dz in [-box_size, 0, box_size]:
                if dx == dy == dz == 0:
                    continue
                shift = np.array([dx, dy, dz])
                replicas.append(points + shift)
    return np.vstack((points, *replicas))

# generate random points in the box
np.random.seed(42) # fix random seeds
points = np.random.rand(numBeads, 3) * box_size  # Generate some random points

# Replicate points to account for PBC
points_PBC = replicate_points(points, box_size)

# Compute the Voronoi tessellation and Delaunay tessellation
tetra = Delaunay(points_PBC)
vor = Voronoi(points_PBC)

In [63]:
vertexSimplex = defaultdict(set)
for tid, simplex in enumerate(tetra.simplices):
    for vid in simplex:
        vertexSimplex[vid].add(tid)

In [64]:
def triangle_area(p, q, r):
    return 0.5 * np.linalg.norm(np.cross(q - p, r - p))

def tetrahedron_incenter(A, B, C, D):
    # Areas of faces opposite each vertex
    alpha_A = triangle_area(B, C, D)  # face BCD
    alpha_B = triangle_area(A, C, D)  # face ACD
    alpha_C = triangle_area(A, B, D)  # face ABD
    alpha_D = triangle_area(A, B, C)  # face ABC

    # Compute weighted sum of vertices
    numerator = (alpha_A * A
                 + alpha_B * B
                 + alpha_C * C
                 + alpha_D * D)
    denominator = (alpha_A + alpha_B + alpha_C + alpha_D)

    # Return the incenter
    return numerator / denominator

comAll = [] # array of COM
for i in range(len(tetra.simplices)):
    point = points_PBC[tetra.simplices[i]]
    comAll.append(tetrahedron_incenter(point[0], point[1], point[2], point[3]))
comAll = np.array(comAll)

In [65]:
primaryStates = set()
# if the COM of the tetra is inside the main box
for i, simplex in enumerate(tetra.simplices):
    com = comAll[i]
    if np.max(com) < box_size and np.min(com) > 0:
        primaryStates.add(i)

In [66]:
# tetra index in the primary cell
def findPrimaryTetra(i):
    global tetra, primaryStates, numBeads, vertexSimplex, indexPrimary

    # if it is already inside
    if i in primaryStates:
        return i
    
    vertices = copy.deepcopy(tetra.simplices[i])
    for i in range(4):
        while vertices[i] >= numBeads:
            vertices[i] -= numBeads
    if len(set(vertices)) < 4:
        return -1
    
    for a in range(27):
        set1 = vertexSimplex[vertices[0]]
        for b in range(27):
            set2 = set1 & vertexSimplex[vertices[1]]
            if len(set2) > 0:
                for c in range(27):
                    set3 = set2 & vertexSimplex[vertices[2]]
                    if len(set3) > 0:
                        for d in range(27):
                            setAll = set3 & vertexSimplex[vertices[3]]
                            if len(setAll) > 0:
                                candidate = setAll.pop()
                                if candidate in indexPrimary:
                                    return indexPrimary[candidate]
                                if candidate in primaryStates:
                                    return candidate

                            vertices[3] += numBeads
                    vertices[2] += numBeads
                    vertices[3] %= numBeads
            vertices[1] += numBeads
            vertices[2] %= numBeads
            vertices[3] %= numBeads
        vertices[0] += numBeads
        vertices[1] %= numBeads
        vertices[2] %= numBeads
        vertices[3] %= numBeads

    return -1

indexPrimary = dict()
for i in range(len(tetra.simplices)):
    indexPrimary[i] = findPrimaryTetra(i)

Counter(Counter(indexPrimary.values()).values())

Counter({27: 4699, 18: 1795, 12: 244, 8: 9, 17585: 1})

In [67]:
# construct graph and edge list
def circumRadius(p1, p2, p3):
    a = np.linalg.norm(p2 - p3)
    b = np.linalg.norm(p1 - p3)
    c = np.linalg.norm(p1 - p2)
    
    cross_product = np.cross(p2 - p1, p3 - p1)
    area = np.linalg.norm(cross_product) / 2

    return (a * b * c) / (4 * area)

def point_to_segment_distance(P, A, B):
    AB = B - A
    normAB2 = np.dot(AB, AB)
    if np.allclose(AB, 0):
        distances = np.linalg.norm(P - A, axis=1)
        return distances
    
    AP = P - A  # shape: (N, d)
    t = np.dot(AP, AB) / normAB2  # shape: (N,)
    t_clamped = np.clip(t, 0, 1)
    closest_points = A + np.outer(t_clamped, AB)
    return np.linalg.norm(P - closest_points, axis=1)

def constructGraph(vor):
    """Filter edges based on distance to the nearest input point."""
    N = len(vor.vertices)
    G = nx.Graph()
    edgeList = []
    visited = set()
    for i, ridge in enumerate(vor.ridge_vertices): # loop over faces
        for vertex_idx in range(len(ridge)-1):     # loop over vertices on face
            v1, v2 = ridge[vertex_idx], ridge[vertex_idx + 1]
            if v1 < 0 or v2 < 0 or (v1, v2) in visited or (v2, v1) in visited:
                continue  # Skip if edge goes to infinity
            visited.add((v1, v2))
            point1 = vor.vertices[v1]
            point2 = vor.vertices[v2]
            if point1[0] < -box_size/2 or point1[1] < -box_size/2 or point1[2] < -box_size/2 or point1[0] > 1.5*box_size or point1[1] > 1.5*box_size or point1[2] > 1.5*box_size:
                continue
            if point2[0] < -box_size/2 or point2[1] < -box_size/2 or point2[2] < -box_size/2 or point2[0] > 1.5*box_size or point2[1] > 1.5*box_size or point2[2] > 1.5*box_size:
                continue

            dist = min(point_to_segment_distance(vor.points, point1, point2))
            G.add_edge(v1, v2)
            edgeList.append([v1, v2, dist])

    print("Number of all nodes: ", N)
    print("Number of nodes: ", len(G.nodes()))
    print("Number of edges: ", len(edgeList))
    return G, edgeList

fullGraph, edgeList = constructGraph(vor)
edgeList.sort(key = lambda x: x[2])

Number of all nodes:  179734
Number of nodes:  53972
Number of edges:  103062


In [68]:
def ifPercolate(component, box_size, vor):
    largestVoidVertices = vor.vertices[np.array(list(component[0]))]
    visited = set()
    for i, vi in enumerate(largestVoidVertices):
        dists = np.linalg.norm(vor.points - vi, axis=1)
        closest_idx = np.argpartition(dists, 4)[:4]
        closest_idx = closest_idx[np.argsort(dists[closest_idx])] % numBeads
        closePoints = tuple(closest_idx)
        if closePoints in visited:
            return True
        else:
            visited.add(closePoints)
    return False

In [69]:
l = len(edgeList)//2
r = len(edgeList)
m = 0
while r-l>1:
    G = copy.deepcopy(fullGraph)
    m = (l+r) // 2
    for i in range(m):
        G.remove_edge(edgeList[i][0], edgeList[i][1])
    print(m, edgeList[m][2])
    connected_components = list(nx.connected_components(G))
    connected_components = [[component,len(component)] for component in connected_components]
    connected_components.sort(key = lambda x:x[1], reverse=True)
    ifPerco = False
    for component in connected_components[:5]:
        if ifPercolate(component,box_size, vor):
            ifPerco = True
            break
    if len(connected_components) != 0 and ifPerco:
        l = m
    else:
        r = m
mPerco = m

77296 9.181365355832503
90179 10.067674591586108
83737 9.586139242592399
80516 9.388578455480388
78906 9.282399617890334
78101 9.235844619212088
77698 9.199469905578134
77497 9.19171431113398
77597 9.195242088792218
77647 9.197034584653899
77622 9.196449542368965
77609 9.195481806050077
77615 9.196438094639507
77612 9.19548180605008
77613 9.19548180605008


In [70]:
rPerco = edgeList[m][2]
r_cut = rPerco
G = copy.deepcopy(fullGraph)
for i in range(len(edgeList)):
    if edgeList[i][2] > r_cut:
        break
    G.remove_edge(edgeList[i][0], edgeList[i][1])

connected_components = list(nx.connected_components(G))
connected_components = [[component,len(component)] for component in connected_components]
connected_components.sort(key = lambda x:x[1], reverse=True)
ifPerco = False
for component in connected_components[:5]:
    if ifPercolate(component, box_size, vor):
        ifPerco = True
        break
if not ifPerco:
    component = connected_components[0]

print(r_cut, ifPercolate(component, box_size, vor), len(list(component[0])))

9.19548180605008 False 1411


### Build Markov Chain

In [71]:
subG = G.subgraph(component[0])
vertices = vor.vertices[list(component[0])]
states = set()

def segment_intersects_triangle(ABC, P, Q): # if a line segment pass through a triangle
    A, B, C = ABC[0], ABC[1], ABC[2]
    N = np.cross(B - A, C - A)
    norm_N = np.linalg.norm(N)
    d = Q - P
    denom = np.dot(d, N)
    t = -np.dot(P - A, N) / denom
    if t < 0 or t > 1:
        # Intersection point is not on the line segment.
        return False
    
    # barycentric coordinates: check if Intersection point is inside triangle
    X = P + t * d
    v0 = C - A
    v1 = B - A
    v2 = X - A
    dot00 = np.dot(v0, v0)
    dot01 = np.dot(v0, v1)
    dot02 = np.dot(v0, v2)
    dot11 = np.dot(v1, v1)
    dot12 = np.dot(v1, v2)
    invDenom = 1.0 / (dot00 * dot11 - dot01 * dot01)
    u = (dot11 * dot02 - dot01 * dot12) * invDenom
    v = (dot00 * dot12 - dot01 * dot02) * invDenom
    if u >= 0 and v >= 0 and (u + v) <= 1.0:
        return True
    else:
        return False
    
def bisectionSearch(p1, p2):
    global edge

    tetra1 = int(tetra.find_simplex(p1))
    tetra2 = int(tetra.find_simplex(p2))
    states.add(tetra1)
    states.add(tetra2)

    if tetra1 == tetra2:
        return
    
    # deal with werid case
    # 1. edges from VT may pass through some tetrahedron. So the two ends of the edge are not in neighboring tetrahedrons.
    # 2. edges from VT may pass through other tetrahedron and then end up in the neighboring tetrahedron.
    commonVertices = set(tetra.simplices[tetra1]) & set(tetra.simplices[tetra2])
    if len(commonVertices) < 3 or not segment_intersects_triangle(points_PBC[list(commonVertices)], p1, p2):
        midPoint = (p1 + p2) / 2
        bisectionSearch(p1, midPoint)
        bisectionSearch(midPoint, p2)

for edge in list(subG.edges):
    bisectionSearch(vor.vertices[edge[0]], vor.vertices[edge[1]])
    t1 = int(tetra.find_simplex(vor.vertices[edge[0]]))
    t2 = int(tetra.find_simplex(vor.vertices[edge[1]]))
        
states = list(states)
states.sort()

statePrimary = set() # id of the primary states
for i in states:
    statePrimary.add(indexPrimary[i])
if -1 in statePrimary:
    statePrimary.remove(-1)
    
statePrimary = list(statePrimary) # statePrimary[new id] = original id
statePrimary.sort()
nStates = len(statePrimary)
print("Number of primary states = ",nStates)

statePrimaryInverse = {value: key for key, value in enumerate(statePrimary)} # statePrimaryInverse[original id] = new id

Number of primary states =  830


In [72]:
# compute volume of tetra
def sample_point_in_tetrahedron(vertices):
    rn = -np.log(np.random.rand(4))
    return np.dot(rn / np.sum(rn), vertices)

nTrails = 10_000
def cavityVolume(vertices, r):
    global nTrials
    V0 = abs(np.dot(vertices[1]-vertices[0], np.cross(vertices[2]-vertices[0], vertices[3]-vertices[0]))) / 6.0

    cnt = 0
    for _  in range(nTrails):
        pt = sample_point_in_tetrahedron(vertices)
        for i in range(4):
            if np.linalg.norm(pt - vertices[i]) < r:
                cnt += 1
                break
    if cnt == nTrails:
        cnt -= 1
    return V0 * (nTrails - cnt) / nTrails


idVol = np.zeros(nStates)
for cnt, i in enumerate(statePrimary):
    idVol[cnt] = cavityVolume(points_PBC[tetra.simplices[i]], r_cut)

In [73]:
def distance_to_bisector_intersection(A, B, C):
    """
    AB cross AC bisector at X, make sure AB is the longest side
    return X and AX distance
    """
    AB = B - A
    AC = C - A
    
    # Parameter t along AB for the intersection:
    t = np.linalg.norm(AC)**2 / (2 * np.dot(AC, AB))
    #print(np.linalg.norm(AB), np.linalg.norm(AC), np.linalg.norm(B-C))
    
    # Intersection point:
    X = A + t * AB
    # Distance from A to X:
    d = np.linalg.norm(X - A)
    return d, X

def sphere_line_intersection_closer_to_B(A, B, C, r):
    d = B - A
    V = A - C
    d_norm_sq = np.dot(d, d)
    
    discriminant = (np.dot(V, d))**2 - d_norm_sq*(np.dot(V, V) - r**2)
    sqrt_disc = np.sqrt(discriminant)
    t = (-np.dot(V, d) + sqrt_disc) / d_norm_sq
    
    X = A + t * d
    distance = np.linalg.norm(X - A)
    
    return X, distance

def computeExitArea(p1, p2, p3, r):
    c = np.linalg.norm(p1-p2)
    b = np.linalg.norm(p1-p3)
    a = np.linalg.norm(p3-p2)

    # make sure c is the longest
    if a > b and a > c:
        a, c = copy.deepcopy(c), copy.deepcopy(a)
        p1, p3 = copy.deepcopy(p3), copy.deepcopy(p1)
    if b > a and b > c:
        b, c = copy.deepcopy(c), copy.deepcopy(b)
        p2, p3 = copy.deepcopy(p3), copy.deepcopy(p2)
    s = (a + b + c) / 2
    areaTriangle = np.sqrt(s * (s - a) * (s - b) * (s - c))

    R = a*b*c/4/areaTriangle
    if R <= r:
        return 0
    
    # if obtuse triangle and two circles intersect outside the triangle
    d_AC, X_AC = distance_to_bisector_intersection(p1, p2, p3)
    d_BC, X_BC = distance_to_bisector_intersection(p2, p1, p3)
    #print(d_AC, d_BC, r)
    if r > d_AC and r > d_BC:
        return 0.0
    if r > d_AC or r > d_BC:
        if d_BC < d_AC: # make sure the side close to A is covered instead of B.
            a, b = b, a
            p1, p2 = p2, p1
            d_AC, d_BC = d_BC, d_AC
            X_AC, X_BC = X_BC, X_AC
            X, XA = sphere_line_intersection_closer_to_B(p1, p2, p3, r)
            b = r
            c = c - XA
            s = (a + b + c) / 2
            areaTriangle = np.sqrt(s * (s - a) * (s - b) * (s - c))
            angleX = np.arccos(np.clip(np.dot(p3-X, p2-X) / b / c, -1, 1))
            areaTriangle -= r*r/2 * (np.pi - angleX)
            if a < 2*r:
                areaTriangle += r*r*np.arccos(a / (2 * r)) - (a / 4) * np.sqrt(4 * r**2 - a**2)
            return areaTriangle

    dA = areaTriangle * 2.0 / a
    dB = areaTriangle * 2.0 / b
    dC = areaTriangle * 2.0 / c
    areaTriangle -= np.pi * r * r / 2
    # deal with overlaps
    if a < 2*r:
        areaTriangle += r*r*np.arccos(a / (2 * r)) - (a / 4) * np.sqrt(4 * r**2 - a**2)
    if b < 2*r:
        areaTriangle += r*r*np.arccos(b / (2 * r)) - (b / 4) * np.sqrt(4 * r**2 - b**2)
    if c < 2*r:
        areaTriangle += r*r*np.arccos(c / (2 * r)) - (c / 4) * np.sqrt(4 * r**2 - c**2)
    # deal with truncation from the opsite side
    if dA < r:
        areaTriangle += r**2 * np.arccos(dA/r) - dA * np.sqrt(r**2 - dA**2)
    if dB < r:
        areaTriangle += r**2 * np.arccos(dB/r) - dB * np.sqrt(r**2 - dB**2)
    if dC < r:
        areaTriangle += r**2 * np.arccos(dC/r) - dC * np.sqrt(r**2 - dC**2)

    return areaTriangle

In [74]:
def periodicSearch(com, comDict, box_size):
    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            for dz in [-1, 0, 1]:
                comTmp = (com[0] + dx*box_size, com[1] + dy*box_size, com[2] + dz*box_size)
                if comTmp in comDict:
                    return comDict[comTmp]
    return -1

transitionPrimary = set()
G = nx.DiGraph()
for i in statePrimary:
    neighborTmp = []
    for neighbor in tetra.neighbors[i]:
        if indexPrimary[neighbor] in statePrimary:
            neighborVertices = points_PBC[tetra.simplices[neighbor]]
            vCommon = list(set(tetra.simplices[i]) & set(tetra.simplices[neighbor]))
            exitArea = computeExitArea(points_PBC[vCommon[0]], points_PBC[vCommon[1]], points_PBC[vCommon[2]], r_cut)
            if exitArea > 0.0:
                transitionPrimary.add(i)
                transitionPrimary.add(indexPrimary[neighbor])
                neighborTmp.append(indexPrimary[neighbor])
                G.add_edge(i, indexPrimary[neighbor])
            
print("States not accessible: ", sorted(list((set(statePrimary) - transitionPrimary))))
print("States not accessible: ", sorted(list((transitionPrimary - set(statePrimary)))))

States not accessible:  []
States not accessible:  []


In [75]:
# test unidirectional edge
unidir_edges = [(u, v) for u, v in G.edges() if not G.has_edge(v, u)]
print("Unidirectional edges:", unidir_edges)

Unidirectional edges: []


In [76]:
sccs = list(nx.weakly_connected_components(G))
sccs = sorted(sccs, key=len, reverse = True)
sccs = [list(comp) for comp in sccs]
len(sccs)

1

In [77]:
points_id = [i for i in range(numBeads * 27)]
points_image_id = [i%numBeads for i in range(numBeads * 27)]
points_PBC_df = pd.DataFrame({'id': points_id, 'image_id': points_image_id})

points_PBC_df['tetra_id'] = [[] for _ in range(len(points_PBC_df))]
for tid, simplex in enumerate(tetra.simplices):
    for vid in simplex:
        points_PBC_df.loc[vid, 'tetra_id'].append(tid)

In [78]:
tetra_id = [i for i in range(len(tetra.simplices))]
tetra_obstacles_id = [sorted(i) for i in tetra.simplices]

tetra_df = pd.DataFrame({'id': tetra_id, 'obstacles_id': tetra_obstacles_id})


tetra_df['x'] = comAll[:,0]
tetra_df['y'] = comAll[:,1]
tetra_df['z'] = comAll[:,2]

tetra_df['is_prime'] = (tetra_df['x'] > 0) & (tetra_df['x'] < box_size) & (tetra_df['y'] > 0) & (tetra_df['y'] < box_size) & (tetra_df['z'] > 0) & (tetra_df['z'] < box_size)
tetra_df['image_id'] = tetra_df['id'].apply(lambda x: indexPrimary[x])

tetra_df['is_percolation'] = tetra_df['id'].apply(lambda x: x in statePrimary)
tetra_df['percolation_id'] = tetra_df['is_percolation'].cumsum()
tetra_df.loc[~tetra_df['is_percolation'], 'percolation_id'] = -1
tetra_df.loc[tetra_df['is_percolation'], 'percolation_id'] = tetra_df.loc[tetra_df['is_percolation'], 'percolation_id']-1
tetra_df.loc[tetra_df['is_percolation'], 'volume'] = tetra_df.loc[tetra_df['is_percolation'], 'percolation_id'].apply(lambda x: idVol[x])
tetra_df['vol_fraction'] = tetra_df['volume'] / sum(idVol)
tetra_df['vol_fraction'] = tetra_df['vol_fraction'].fillna(0)
tetra_df['vol_fraction_cum'] = tetra_df['vol_fraction'].cumsum()

## Generate probe particles

In [ ]:
'''
tetra_df['num_probes'] = tetra_df['vol_fraction'].apply(lambda x: int(x*numProbes)+1)
tetra_df.loc[~tetra_df['is_percolation'], 'num_probes'] = 0

probes_XYZ = []
for i, row in tetra_df[tetra_df['is_percolation']].iterrows():
    print(i, row['volume'])
    vertices = np.array([points_PBC[i] for i in row['obstacles_id']])

    cnt = 0
    for _ in range(row['num_probes']):
        ifOverlap = True
        while ifOverlap:
            cnt += 1
            if cnt > int(1e4):
                break
            ifOverlap = False
            probeTmp = sample_point_in_tetrahedron(vertices)
            
            for obstacle in points_PBC:
                if np.linalg.norm(probeTmp - obstacle) <= r_cut:
                    ifOverlap = True
                    break
                
        if not ifOverlap:
            probes_XYZ.append(probeTmp)
probes_XYZ = np.array(probes_XYZ)
'''

In [104]:
probes_XYZ = []
for _ in range(numProbes):
    ifOverlap = True
    while ifOverlap:
        ifOverlap = False
        
        rd = np.random.random()
        ind = tetra_df.loc[tetra_df['vol_fraction_cum']>rd].iloc[0]['id']
        vertices = np.array([points_PBC[i] for i in tetra_df.loc[ind, 'obstacles_id']])
        probeTmp = sample_point_in_tetrahedron(vertices)
        
        for obstacle in points_PBC:
            if np.linalg.norm(probeTmp - obstacle) <= r_cut:
                ifOverlap = True
                break

    probes_XYZ.append(probeTmp)

KeyboardInterrupt: 

## save the results

In [85]:
lengthUnit = r_cut*0.5

tetra_df['x'] = tetra_df['x']/lengthUnit
tetra_df['y'] = tetra_df['y']/lengthUnit
tetra_df['z'] = tetra_df['z']/lengthUnit

tetra_df['volume'] = tetra_df['volume']/lengthUnit**3

In [93]:
# save configurations
outfile = filepath + '.pos'
with open(outfile, 'w') as output_fileID:
    output_fileID.write(f'{len(points) + len(probes_XYZ)}\n')
    output_fileID.write(f'Lattice="{box_size/lengthUnit} 0 0 0 {box_size/lengthUnit} 0 0 0 {box_size/lengthUnit}" Properties=species:S:1:pos:R:3:radius:R:1\n')

    for bid in range(len(points)):
        output_fileID.write(f'{0} {points[bid][0]/lengthUnit} {points[bid][1]/lengthUnit} {points[bid][2]/lengthUnit} {2.0}\n')
    for probe in probes_XYZ:
        output_fileID.write(f'{1} {probe[0]/lengthUnit} {probe[1]/lengthUnit} {probe[2]/lengthUnit} {0.0}\n')

print('Done!')

Done!


In [94]:
# save obstacles info
outfile = filepath + '_obstacles.csv'
#points_PBC_df.to_csv(outfile, index = False, float_format='%.17f')

outfile = filepath + '_tetra.txt'

tetra_df[tetra_df['is_prime']].to_csv(outfile, sep=',', index = False, float_format='%.18f')